In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import time
from mrcnn.config import Config
from datetime import datetime

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# Root directory of the project
ROOT_DIR = os.getcwd()
 
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
 
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "logs","mask_rcnn_polyp_0055.h5")
    
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the polyp dataset.
    """
    # Give the configuration a recognizable name
    NAME = "polyp"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # background + 1 shapes

    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    # anchors size should be the same size while training
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 100

    STEPS_PER_EPOCH = 200   
    VALIDATION_STEPS = 50

config = ShapesConfig()
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained
model.load_weights(COCO_MODEL_PATH, by_name=True)
 
# Class names firsth one should be BG background
class_names = ['BG','polyp','clip']


class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

In [ ]:
#rewrite some function
class PolypDataset(utils.Dataset):
    # Get how many instances (objects) there are in this figure
    def get_obj_index(self, image):
        n = np.max(image)
        return n

    # Parse the yaml file obtained from labelme to get the instance labels for each layer of the mask
    def from_yaml_get_class(self, image_id):
        info = self.image_info[image_id]
        with open(info['yaml_path']) as f:
            temp = yaml.load(f.read())
            labels = temp['label_names']
            del labels[0]
        return labels

    # Rewrite draw_mask
    def draw_mask(self, num_obj, mask, image, image_id):
        # print("draw_mask-->",image_id)
        # print("self.image_info",self.image_info)
        info = self.image_info[image_id]
        # print("info-->",info)
        # print("info[width]----->",info['width'],"-info[height]--->",info['height'])
        for index in range(num_obj):
            for i in range(info['width']):
                for j in range(info['height']):
                    # print("image_id-->",image_id,"-i--->",i,"-j--->",j)
                    # print("info[width]----->",info['width'],"-info[height]--->",info['height'])
                    at_pixel = image.getpixel((i, j))
                    if at_pixel == index + 1:
                        mask[j, i, index] = 1
        return mask

    # Rewrite load_shapes to include your own categories
    # and added path, mask_path, yaml_path to self.image_info information
    def load_shapes(self, count, img_folder, mask_folder, imglist, dataset_root_path, yaml_folder):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("polyp", 1, "polyp")
        self.add_class("polyp", 2, "clip")
        #self.add_class("shapes", 3, "hole")

        for i in range(count):
            # Get image width and height
            print(i)
            filestr = imglist[i].split(".")[0]
            # print(imglist[i],"-->",cv_img.shape[1],"--->",cv_img.shape[0])
            # print("id-->", i, " imglist[", i, "]-->", imglist[i],"filestr-->",filestr)
            # filestr = filestr.split("_")[1]
            mask_path = mask_folder + "/" + imglist[i]
            print(mask_path)
            yaml_path = yaml_folder + "/" + filestr + ".yaml"
            print(img_folder + "/" + imglist[i])
            cv_img = cv2.imread(img_folder + "/" + imglist[i])
            self.add_image("polyp", image_id=i, path=img_folder + "/" + imglist[i],
                           width=cv_img.shape[1], height=cv_img.shape[0], mask_path=mask_path, yaml_path=yaml_path)


    # Rewrite load_mask
    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        global iter_num
        print("image_id", image_id)
        info = self.image_info[image_id]
        count = 1  # number of object
        img = Image.open(info['mask_path'])
        num_obj = self.get_obj_index(img)
        mask = np.zeros([info['height'], info['width'], num_obj], dtype=np.uint8)
        mask = self.draw_mask(num_obj, mask, img, image_id)
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(count - 2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion

            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        labels = []
        labels = self.from_yaml_get_class(image_id)

        labels_form = []
        for i in range(len(labels)):
            if labels[i].find("polyp") != -1:            #add your class here
                labels_form.append("polyp")
            elif labels[i].find("clip") != -1:
                labels_form.append("clip")
            #elif labels[i].find("hole") != -1:
            #    labels_form.append("hole")

        class_ids = np.array([self.class_names.index(s) for s in labels_form])
        return mask, class_ids.astype(np.int32)


def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size * cols, size * rows))
    return ax

In [ ]:
dataset_root_path = "./"
img_folder = dataset_root_path + "../img_folder/test"
mask_folder = dataset_root_path + "../mask_folder/test" 
yaml_folder = dataset_root_path + "../yaml_folder/test"
imglist = os.listdir(img_folder)
count = len(imglist)

# train and val dataset preparation
dataset_test = PolypDataset()
dataset_test.load_shapes(count, img_folder, mask_folder, imglist, dataset_root_path,yaml_folder)
dataset_test.prepare()

In [ ]:
from PIL import Image
import yaml
# Load and display random samples
image_ids = np.random.choice(dataset_test.image_ids, 4)
for image_id in image_ids:
    image = dataset_test.load_image(image_id)
    mask, class_ids = dataset_test.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_test.class_names)

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
image_ids = dataset_test.image_ids
APs = []
F1_scores = []
total_recalls = []
total_precisions = []
ious=[]
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    
    total_recalls.append(np.mean(recalls))
    total_precisions.append(np.mean(precisions))
    F1_scores.append((2* (np.mean(precisions) * np.mean(recalls)))/(np.mean(precisions) + np.mean(recalls)))
    APs.append(AP)
    ious.append(np.nan_to_num(np.mean(overlaps).astype(np.float64)))
    
print("mAP: ", np.mean(APs))
print("recalls: ", np.mean(total_recalls))
print("precisions: ", np.mean(total_precisions))
print("F1_scores: ", np.mean(F1_scores))
print("IOUs: ", np.mean(ious))